# 1次元拡散方程式

In [117]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [118]:
# init
mu = 0.04  # 熱拡散係数 [m^2/s]  銅 0.117, 鉄0.022
dt = 0.001  # [sec]
dx = 0.01  # [m]

tnum = 10

xmin, xmax = 0, 1
xnum = int((xmax - xmin) / dx + 1)
xs = np.linspace(xmin, xmax, xnum)
us = np.zeros((tnum, xnum))

In [119]:
# 安定性条件
d = mu * dt / dx**2
assert d < 1/2, d

In [120]:
# 初期条件
# 全体がinit_temp度x=1.0で恒温
init_temp = 60
us[0, :] = init_temp

# パイプが沈み込む速度(等速)
v_pipe = 10  # [m/s]

# 時刻1で先が0度になる
us[1, 0] = 0

In [121]:
def plot(us, it):
    plt.plot(xs, us[it, :])
    plt.xlabel("x")
    plt.ylabel("temperature")

def save_fig(us, it, path, ice_h):
    fig = plt.figure(facecolor="white", figsize=(5, 8))
    plt.plot(us[it, :], xs)
    plt.xlabel("temperature")
    plt.axvspan(-1, 100, 0, ice_h, color = "coral")
    plt.ylabel("x")
    plt.ylim(0, 1.0)
    plt.title("t={0} [s]".format(np.round(it * dt, 10)))
    plt.xlim(-1, 100)
    fig.savefig(path)
    plt.close()

In [122]:
for it in tqdm(range(0, tnum)):
    us[it, -1] = init_temp  # x=1.0で恒温

    # パイプが沈み込む効果
    x_bound = v_pipe * dt * it  # 氷の位置
    is_ice = xs < x_bound
    us[it-1, is_ice] = 0.0
    
    if it != 0:
        for ix in range(1, xnum-1):
            us[it, ix] = us[it-1, ix] + d * (us[it-1, ix+1] - 2 * us[it-1, ix] + us[it-1, ix-1])
    save_fig(us=us, it=it, path="img/it={0:03}".format(it), ice_h=x_bound)

100%|██████████| 10/10 [00:00<00:00, 12.45it/s]


In [123]:
# animation
from PIL import Image
import glob

files = sorted(glob.glob('img/*.png'))  
images = list(map(lambda file : Image.open(file) , files))
images[0].save('animation.gif', save_all=True, \
    append_images=images[1:], fps=100, loop=0)

## 一定速度で沈み込んだ時